## Библиотеки

In [64]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf

from sklearn import linear_model
import statsmodels.formula.api as smf
import sklearn.model_selection as skm_s
from sklearn.metrics import classification_report, confusion_matrix, roc_curve
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import math
from sklearn.model_selection import StratifiedShuffleSplit

## Загрузка данных

In [65]:
df = pd.read_csv('lemurs.csv', sep=',', encoding = "ISO-8859-1")
df

,Sex,Weight,Age,Preg_Status,Birth_Type
0,0,1902.0,2.12,0,0
1,0,2447.0,2.38,0,0
2,0,2306.0,3.47,0,0
3,0,2446.5,4.02,0,0
4,0,2319.5,6.53,0,0
...,...,...,...,...,...
3212,1,540.0,0.23,0,1
3213,1,820.0,0.32,0,1
3214,1,1120.0,0.42,0,1
3215,1,1320.0,0.65,0,1


In [66]:
df = df.dropna()
df = df.loc[df['Age'] >= 3]
df

,Sex,Weight,Age,Preg_Status,Birth_Type
2,0,2306.0,3.47,0,0
3,0,2446.5,4.02,0,0
4,0,2319.5,6.53,0,0
5,0,2283.5,7.92,0,0
6,0,2100.0,9.37,0,0
...,...,...,...,...,...
3101,1,2560.0,3.81,0,1
3102,1,2480.0,3.90,0,1
3103,1,2540.0,4.00,0,1
3104,1,2540.0,4.23,0,1


## Линейная

In [67]:
model = smf.ols(formula="Weight ~ Sex + Age + Preg_Status + Birth_Type", data=df).fit()

print(model.summary())
print(f'\nMAPE: {round(np.mean(np.abs((df.Weight - model.predict()) / df.Weight)) * 100, 2)}%\n')

                            OLS Regression Results                            
Dep. Variable:                 Weight   R-squared:                       0.139
Model:                            OLS   Adj. R-squared:                  0.138
Method:                 Least Squares   F-statistic:                     90.31
Date:                Thu, 22 Dec 2022   Prob (F-statistic):           3.14e-71
Time:                        17:42:19   Log-Likelihood:                -15706.
No. Observations:                2234   AIC:                         3.142e+04
Df Residuals:                    2229   BIC:                         3.145e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept    2228.2771     23.590     94.458      

## Логарифм +

In [68]:
model1 = smf.ols(formula="np.log(Weight) ~ Sex + Age + Preg_Status + Birth_Type", data=df).fit()

print(model1.summary())
print(f'\nMAPE: {round(np.mean(np.abs((np.log(df.Weight) - model1.predict()) / np.log(df.Weight))) * 100, 2)}%\n')

                            OLS Regression Results                            
Dep. Variable:         np.log(Weight)   R-squared:                       0.136
Model:                            OLS   Adj. R-squared:                  0.134
Method:                 Least Squares   F-statistic:                     87.52
Date:                Thu, 22 Dec 2022   Prob (F-statistic):           3.73e-69
Time:                        17:42:19   Log-Likelihood:                 1756.9
No. Observations:                2234   AIC:                            -3504.
Df Residuals:                    2229   BIC:                            -3475.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept       7.7101      0.010    811.222      

## Логарифм *

In [69]:
model2 = smf.ols(formula="np.log(Weight) ~ Sex * Age * Preg_Status * Birth_Type", data=df).fit()

print(model2.summary())
print(f'\nMAPE: {round(np.mean(np.abs((np.log(df.Weight) - model2.predict()) / np.log(df.Weight))) * 100, 2)}%\n')

                            OLS Regression Results                            
Dep. Variable:         np.log(Weight)   R-squared:                       0.145
Model:                            OLS   Adj. R-squared:                  0.141
Method:                 Least Squares   F-statistic:                     34.24
Date:                Thu, 22 Dec 2022   Prob (F-statistic):           4.84e-68
Time:                        17:42:19   Log-Likelihood:                 1768.8
No. Observations:                2234   AIC:                            -3514.
Df Residuals:                    2222   BIC:                            -3445.
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                                     coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------
Intercept   

## Прогноз по среднему значению

In [70]:
data = {
    'Sex': round(np.mean(df['Sex'])),
    'Age': round(np.mean(df['Age'])),
    'Birth_Type': round(np.mean(df['Birth_Type'])),
    'Preg_Status': round(np.mean(df['Preg_Status']))
}

mean_df = pd.DataFrame(data, index=[0])
mean_df

,Sex,Age,Birth_Type,Preg_Status
0,0,11,1,0


In [71]:
pred = df.loc[df['Sex'] == 0]
pred = pred.loc[round(pred['Age']) == 10]
pred = pred.loc[pred['Birth_Type'] == 1]
pred = pred.loc[pred['Preg_Status'] == 0]
pred

,Sex,Weight,Age,Preg_Status,Birth_Type
188,0,2540.0,9.75,0,1
189,0,2520.0,9.76,0,1
190,0,2540.0,9.78,0,1
265,0,2380.0,9.52,0,1
266,0,2280.0,9.67,0,1
...,...,...,...,...,...
2138,0,2240.0,10.06,0,1
2139,0,2200.0,10.19,0,1
2140,0,2400.0,10.43,0,1
2306,0,2320.0,9.53,0,1


### Прогноз линейной

In [72]:
pr = round(model.predict(mean_df)[0], 2)
rl = round(np.median(pred.Weight), 2)

print(f'Predict: {pr}')
print(f'Real: {rl}')
print(f'Dif: {abs(round(pr - rl, 2))}')

Predict: 2411.95
Real: 2420.0
Dif: 8.05


### Прогноз логарифм +

In [73]:
pr = round(math.e ** model1.predict(mean_df)[0], 2)
rl = round(np.median(pred.Weight), 2)

print(f'Predict: {pr}')
print(f'Real: {rl}')
print(f'Dif: {abs(round(pr - rl, 2))}')

Predict: 2397.39
Real: 2420.0
Dif: 22.61


### Прогноз логарифм *

In [74]:
pr = round(math.e ** model2.predict(mean_df)[0], 2)
rl = round(np.median(pred.Weight), 2)

print(f'Predict: {pr}')
print(f'Real: {rl}')
print(f'Dif: {abs(round(pr - rl, 2))}')

Predict: 2397.3
Real: 2420.0
Dif: 22.7


## Прогноз на тестовых данных

In [75]:
x = df[['Preg_Status', 'Age', 'Sex', 'Birth_Type']]
y = df['Weight']

x_train, x_test, y_train, y_test = skm_s.train_test_split(x, y, test_size=0.7)

### Прогноз линейной

In [76]:
predictions = model.predict(x_test)

errors = []

for i in range(len(predictions)):
    pr = round(y_test.values[i])
    rl = round(predictions.values[i])
    dif = abs(pr - rl)

    errors.append(dif)

    # print(f'pr: {pr}, rl: {rl}, dif: {dif}')

print(f'MAPE: {round(np.mean(np.abs((y_test - predictions) / y_test)) * 100, 2)}%')
print(f'Mean dif: {round(np.mean(errors), 2)}')
print(f'Median dif: {round(np.median(errors), 2)}')

MAPE: 8.83%
Mean dif: 216.24
Median dif: 176.0


### Прогноз логарифм +

In [77]:
predictions = model1.predict(x_test)

errors = []

for i in range(len(predictions)):
    pr = round(y_test.values[i])
    rl = round(math.e ** predictions.values[i])
    dif = abs(pr - rl)

    errors.append(dif)

    # print(f'pr: {pr}, rl: {rl}, dif: {dif}')

print(f'MAPE: {round(np.mean(np.abs((np.log(y_test) - predictions) / np.log(y_test))) * 100, 2)}%')
print(f'Mean dif: {round(np.mean(errors), 2)}')
print(f'Median dif: {round(np.median(errors), 2)}')

MAPE: 1.12%
Mean dif: 215.43
Median dif: 174.0


### Прогноз логарифм *

In [78]:
predictions = model2.predict(x_test)

errors = []

for i in range(len(predictions)):
    pr = round(y_test.values[i])
    rl = round(math.e ** predictions.values[i])
    dif = abs(pr - rl)

    errors.append(dif)

    # print(f'pr: {pr}, rl: {rl}, dif: {dif}')

print(f'MAPE: {round(np.mean(np.abs((np.log(y_test) - predictions) / np.log(y_test))) * 100, 2)}%')
print(f'Mean dif: {round(np.mean(errors), 2)}')
print(f'Median dif: {round(np.median(errors), 2)}')

MAPE: 1.11%
Mean dif: 213.55
Median dif: 171.0


## Модель лассо

In [79]:
x = df[['Preg_Status', 'Age', 'Sex', 'Birth_Type']]
y = df['Weight']

lasso_cv_model = linear_model.LassoCV(alphas = np.random.randint(0, 1000, 1000), cv = 5, max_iter = 100000).fit(x, y)
lasso_cv_model.alpha_

c:\Users\theba\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:634: UserWarning: Coordinate descent without L1 regularization may lead to unexpected results and is discouraged. Set l1_ratio > 0 to add L1 regularization.
  model = cd_fast.enet_coordinate_descent_gram(
c:\Users\theba\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:634: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 65390154.88033273, tolerance: 15313.950247229997
  model = cd_fast.enet_coordinate_descent_gram(
c:\Users\theba\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:634: UserWarning: Coordinate descent without L1 regularization may lead to unexpected results and is discouraged. Set l1_ratio > 0 to add L1 regularization.
  model = cd_fast.enet_coordinate_descent_gram(
c:\Users\theba\AppData\

0

In [80]:
clf = linear_model.Lasso(alpha=lasso_cv_model.alpha_)

x = df[['Preg_Status', 'Age', 'Sex', 'Birth_Type']]
y = df['Weight']

clf.fit(x, y)

print(f"Score: {round(clf.score(x, y), 2)}\n")

for i in range(len(clf.coef_)):
    print(f'{clf.feature_names_in_[i]}: {round(clf.coef_[i], 2)}')

Score: 0.14

Preg_Status: 179.95
Age: 3.19
Sex: 199.62
Birth_Type: 148.59


C:\Users\theba\AppData\Local\Temp\ipykernel_2532\987714510.py:6: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  clf.fit(x, y)
c:\Users\theba\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:648: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
c:\Users\theba\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.358e+07, tolerance: 1.942e+04 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  mo

### Прогноз лассо

In [81]:
predictions = clf.predict(x_test)

errors = []

for i in range(len(predictions)):
    pr = round(y_test.values[i])
    rl = round(predictions[i])
    dif = abs(pr - rl)

    errors.append(dif)

    # print(f'pr: {pr}, rl: {rl}, dif: {dif}')

print(f'MAPE: {round(np.mean(np.abs((y_test - predictions) / y_test)) * 100, 2)}%')
print(f'Mean dif: {round(np.mean(errors), 2)}')
print(f'Median dif: {round(np.median(errors), 2)}')

MAPE: 8.83%
Mean dif: 216.24
Median dif: 176.0


### Прогноз линейной

In [82]:
predictions = model.predict(x_test)

errors = []

for i in range(len(predictions)):
    pr = round(y_test.values[i])
    rl = round(predictions.values[i])
    dif = abs(pr - rl)

    errors.append(dif)

    # print(f'pr: {pr}, rl: {rl}, dif: {dif}')

print(f'MAPE: {round(np.mean(np.abs((y_test - predictions) / y_test)) * 100, 2)}%')
print(f'Mean dif: {round(np.mean(errors), 2)}')
print(f'Median dif: {round(np.median(errors), 2)}')

MAPE: 8.83%
Mean dif: 216.24
Median dif: 176.0


## Матрица корреляции

In [83]:
df.corr()

,Sex,Weight,Age,Preg_Status,Birth_Type
Sex,1.000000,0.306112,-0.168013,0.267874,-0.286590
Weight,0.306112,1.000000,-0.026003,0.208402,0.070129
Age,-0.168013,-0.026003,1.000000,-0.088680,-0.122552
Preg_Status,0.267874,0.208402,-0.088680,1.000000,-0.024222
Birth_Type,-0.286590,0.070129,-0.122552,-0.024222,1.000000


## Модель ридж

In [84]:
x = df[['Preg_Status', 'Age', 'Sex', 'Birth_Type']]
y = df['Weight']

ridge_cv_model = linear_model.RidgeCV(alphas = np.random.randint(1, 1000, 1000)).fit(x, y)
ridge_cv_model.alpha_

3

In [85]:
clr = linear_model.Ridge(alpha=ridge_cv_model.alpha_)

x = df[['Preg_Status', 'Age', 'Sex', 'Birth_Type']]
y = df['Weight']

clr.fit(x, y)

print(f"Score: {round(clr.score(x, y), 2)}\n")

for i in range(len(clr.coef_)):
    print(f'{clr.feature_names_in_[i]}: {round(clr.coef_[i], 2)}')

Score: 0.14

Preg_Status: 175.18
Age: 3.14
Sex: 198.52
Birth_Type: 146.37


### Прогноз ридж

In [86]:
predictions = clr.predict(x_test)

errors = []

for i in range(len(predictions)):
    pr = round(y_test.values[i])
    rl = round(predictions[i])
    dif = abs(pr - rl)

    errors.append(dif)

    # print(f'pr: {pr}, rl: {rl}, dif: {dif}')

print(f'MAPE: {round(np.mean(np.abs((y_test - predictions) / y_test)) * 100, 2)}%')
print(f'Mean dif: {round(np.mean(errors), 2)}')
print(f'Median dif: {round(np.median(errors), 2)}')

MAPE: 8.83%
Mean dif: 216.28
Median dif: 176.0


### Прогноз линейной

In [87]:
predictions = model.predict(x_test)

errors = []

for i in range(len(predictions)):
    pr = round(y_test.values[i])
    rl = round(predictions.values[i])
    dif = abs(pr - rl)

    errors.append(dif)

    # print(f'pr: {pr}, rl: {rl}, dif: {dif}')

print(f'MAPE: {round(np.mean(np.abs((y_test - predictions) / y_test)) * 100, 2)}%')
print(f'Mean dif: {round(np.mean(errors), 2)}')
print(f'Median dif: {round(np.median(errors), 2)}')

MAPE: 8.83%
Mean dif: 216.24
Median dif: 176.0


In [88]:
features = ['Preg_Status', 'Age', 'Sex', 'Birth_Type']
x = df.loc[:, features].values
y = df.loc[:,['Weight']].values
x = StandardScaler().fit_transform(x)

In [89]:
pca = PCA(n_components=3)
components = pca.fit_transform(x)
pc = pd.DataFrame(data = components, columns = ['PC1', 'PC2', 'PC3'])

In [90]:
print(pca.components_.transpose(), '\n')

cum_explained_var = []
for i in range(0, len(pca.explained_variance_ratio_)):
    if i == 0:
        cum_explained_var.append(pca.explained_variance_ratio_[i])
    else:
        cum_explained_var.append(pca.explained_variance_ratio_[i] + 
                                 cum_explained_var[i - 1])

print(cum_explained_var)

[[ 0.51324913 -0.25115146  0.75363236]
 [-0.25923248  0.7009993   0.56253369]
 [ 0.70109714  0.04238254 -0.16485567]
 [-0.42170684 -0.6661281   0.29734949]] 

[0.35765177450183444, 0.6421690768469173, 0.8585823455464088]


In [91]:
pca_df = pd.concat([pc, df[['Weight']]], axis = 1)

In [92]:
pca_df = pca_df.dropna()
pca_df

,PC1,PC2,PC3,Weight
2,0.547129,1.087268,-1.213014,2306.0
3,0.485199,1.254736,-1.078625,2446.5
4,0.420595,1.429432,-0.938436,2319.5
5,0.410348,1.457143,-0.916199,2283.5
6,0.403665,1.475215,-0.901696,2100.0
...,...,...,...,...
2219,-0.613224,-0.938392,-0.460757,2360.0
2220,-0.626145,-0.903453,-0.432719,2280.0
2221,-0.634164,-0.881766,-0.415317,2320.0
2222,-0.648867,-0.842008,-0.383411,2320.0


In [93]:
model = smf.ols(formula="Weight ~ PC1 + PC2 + PC3", data=pca_df).fit()

print(model.summary())
print(f'\nMAPE: {round(np.mean(np.abs((pca_df.Weight - model.predict()) / pca_df.Weight)) * 100, 2)}%\n')

                            OLS Regression Results                            
Dep. Variable:                 Weight   R-squared:                       0.044
Model:                            OLS   Adj. R-squared:                  0.043
Method:                 Least Squares   F-statistic:                     29.16
Date:                Thu, 22 Dec 2022   Prob (F-statistic):           1.97e-18
Time:                        17:42:22   Log-Likelihood:                -13521.
No. Observations:                1901   AIC:                         2.705e+04
Df Residuals:                    1897   BIC:                         2.707e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   2496.9574      6.839    365.114      0.0

In [94]:
x = pca_df[['PC1', 'PC2', 'PC3']]
y = pca_df['Weight']

x_train, x_test, y_train, y_test = skm_s.train_test_split(x, y, test_size=0.7)

### Прогноз линейной 

In [95]:
predictions = model.predict(x_test)

errors = []

for i in range(len(predictions)):
    pr = round(y_test.values[i])
    rl = round(predictions.values[i])
    dif = abs(pr - rl)

    errors.append(dif)
    
print(f'MAPE: {round(np.mean(np.abs((y_test - predictions) / y_test)) * 100, 2)}%')
print(f'Mean dif: {round(np.mean(errors), 2)}')
print(f'Median dif: {round(np.median(errors), 2)}')

MAPE: 9.87%
Mean dif: 240.97
Median dif: 211.0


In [96]:
model1 = smf.ols(formula="np.log(Weight) ~ PC1 + PC2 + PC3", data=pca_df).fit()

print(model1.summary())
print(f'\nMAPE: {round(np.mean(np.abs((np.log(pca_df.Weight) - model1.predict()) / np.log(pca_df.Weight))) * 100, 2)}%\n')

                            OLS Regression Results                            
Dep. Variable:         np.log(Weight)   R-squared:                       0.044
Model:                            OLS   Adj. R-squared:                  0.043
Method:                 Least Squares   F-statistic:                     29.34
Date:                Thu, 22 Dec 2022   Prob (F-statistic):           1.53e-18
Time:                        17:42:23   Log-Likelihood:                 1348.7
No. Observations:                1901   AIC:                            -2689.
Df Residuals:                    1897   BIC:                            -2667.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      7.8155      0.003   2851.640      0.0

### Прогноз логарифм +

In [97]:
predictions = model1.predict(x_test)

errors = []

for i in range(len(predictions)):
    pr = round(y_test.values[i])
    rl = round(math.e ** predictions.values[i])
    dif = abs(pr - rl)

    errors.append(dif)


print(f'MAPE: {round(np.mean(np.abs((np.log(y_test) - predictions) / np.log(y_test))) * 100, 2)}%')
print(f'Mean dif: {round(np.mean(errors), 2)}')
print(f'Median dif: {round(np.median(errors), 2)}')

MAPE: 1.24%
Mean dif: 239.76
Median dif: 205.0


In [98]:
model2 = smf.ols(formula="np.log(Weight) ~ PC1 * PC2 * PC3", data=pca_df).fit()

print(model2.summary())
print(f'\nMAPE: {round(np.mean(np.abs((np.log(pca_df.Weight) - model2.predict()) / np.log(pca_df.Weight))) * 100, 2)}%\n')

                            OLS Regression Results                            
Dep. Variable:         np.log(Weight)   R-squared:                       0.084
Model:                            OLS   Adj. R-squared:                  0.081
Method:                 Least Squares   F-statistic:                     24.92
Date:                Thu, 22 Dec 2022   Prob (F-statistic):           1.03e-32
Time:                        17:42:23   Log-Likelihood:                 1389.4
No. Observations:                1901   AIC:                            -2763.
Df Residuals:                    1893   BIC:                            -2718.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept       7.8115      0.004   2154.537      

### Прогноз логарифм *

In [99]:
predictions = model2.predict(x_test)

errors = []

for i in range(len(predictions)):
    pr = round(y_test.values[i])
    rl = round(math.e ** predictions.values[i])
    dif = abs(pr - rl)

    errors.append(dif)

print(f'MAPE: {round(np.mean(np.abs((np.log(y_test) - predictions) / np.log(y_test))) * 100, 2)}%')
print(f'Mean dif: {round(np.mean(errors), 2)}')
print(f'Median dif: {round(np.median(errors), 2)}')

MAPE: 1.2%
Mean dif: 232.44
Median dif: 201.0


### Прогноз линейной с кв

In [100]:
x = pca_df[['PC1', 'PC2', 'PC3']]
y = pca_df['Weight']

means = []
mean_errors = []
median_errors = []

for i in range(5):
    x_train, x_test, y_train, y_test = skm_s.train_test_split(x, y, test_size=0.7)
    train_df = pd.concat([x_train, y_train], axis = 1)
    model = smf.ols(formula="Weight ~ PC1 + PC2 + PC3", data=pca_df).fit()
    mape = round(np.mean(np.abs((y_test - model.predict(x_test)) / y_test)) * 100, 2)
    means.append(mape)

    errors = []

    for i in range(len(model.predict(x_test))):
        pr = round(y_test.values[i])
        rl = round(model.predict(x_test).values[i])
        dif = abs(pr - rl)

        errors.append(dif)
    
    mean_errors.append(np.mean(errors))
    median_errors.append(np.median(errors))

print(f'MAPE: {round(np.mean(means), 2)}%')
print(f'Mean dif: {round(np.mean(mean_errors), 2)}')
print(f'Median dif: {round(np.mean(median_errors), 2)}')

MAPE: 9.79%
Mean dif: 240.97
Median dif: 212.4


### Прогноз логарифм + с кв

In [101]:
x = pca_df[['PC1', 'PC2', 'PC3']]
y = pca_df['Weight']

means = []
mean_errors = []
median_errors = []

for i in range(5):
    x_train, x_test, y_train, y_test = skm_s.train_test_split(x, y, test_size=0.7)
    train_df = pd.concat([x_train, y_train], axis = 1)
    model = smf.ols(formula="np.log(Weight) ~ PC1 + PC2 + PC3", data=pca_df).fit()
    mape = round(np.mean(np.abs((np.log(y_test) - model.predict(x_test)) / np.log(y_test))) * 100, 2)
    means.append(mape)

    errors = []

    for i in range(len(model.predict(x_test))):
        pr = round(y_test.values[i])
        rl = round(math.e ** model.predict(x_test).values[i])
        dif = abs(pr - rl)

        errors.append(dif)
    
    mean_errors.append(np.mean(errors))
    median_errors.append(np.median(errors))

print(f'MAPE: {round(np.mean(means), 2)}%')
print(f'Mean dif: {round(np.mean(mean_errors), 2)}')
print(f'Median dif: {round(np.mean(median_errors), 2)}')

MAPE: 1.23%
Mean dif: 239.15
Median dif: 205.0


### Прогноз логарифм *

In [102]:
x = pca_df[['PC1', 'PC2', 'PC3']]
y = pca_df['Weight']

means = []
mean_errors = []
median_errors = []

for i in range(5):
    x_train, x_test, y_train, y_test = skm_s.train_test_split(x, y, test_size=0.7)
    train_df = pd.concat([x_train, y_train], axis = 1)
    model = smf.ols(formula="np.log(Weight) ~ PC1 * PC2 * PC3", data=pca_df).fit()
    mape = round(np.mean(np.abs((np.log(y_test) - model.predict(x_test)) / np.log(y_test))) * 100, 2)
    means.append(mape)

    errors = []

    for i in range(len(model.predict(x_test))):
        pr = round(y_test.values[i])
        rl = round(math.e ** model.predict(x_test).values[i])
        dif = abs(pr - rl)

        errors.append(dif)
    
    mean_errors.append(np.mean(errors))
    median_errors.append(np.median(errors))

print(f'MAPE: {round(np.mean(means), 2)}%')
print(f'Mean dif: {round(np.mean(mean_errors), 2)}')
print(f'Median dif: {round(np.mean(median_errors), 2)}')

MAPE: 1.2%
Mean dif: 233.98
Median dif: 204.2
